<a href="https://colab.research.google.com/github/tx1103mark/tweet-sentiment/blob/master/TPUs_in_Colab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TPUs in Colab&nbsp; <a href="https://cloud.google.com/tpu/"><img valign="middle" src="https://raw.githubusercontent.com/GoogleCloudPlatform/tensorflow-without-a-phd/master/tensorflow-rl-pong/images/tpu-hexagon.png" width="50"></a>
In this example, we'll work through training a model to classify images of
flowers on Google's lightning-fast Cloud TPUs. Our model will take as input a photo of a flower and return whether it is a daisy, dandelion, rose, sunflower, or tulip.

We use the Keras framework, new to TPUs in TF 2.1.0. Adapted from [this notebook](https://colab.research.google.com/github/GoogleCloudPlatform/training-data-analyst/blob/master/courses/fast-and-lean-data-science/07_Keras_Flowers_TPU_xception_fine_tuned_best.ipynb) by [Martin Gorner](https://twitter.com/martin_gorner).

#### License

Copyright 2019-2020 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.


---


This is not an official Google product but sample code provided for an educational purpose.


## Enabling and testing the TPU

First, you'll need to enable TPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select TPU from the Hardware Accelerator drop-down

Next, we'll check that we can connect to the TPU:

#Data process

In [ ]:
STATUS OnnxInputAdjustOpPass::AdjustResize(const CNodePtr &cnode) {
  MS_ASSERT(cnode != nullptr);
  auto node = cnode->input(0);
  MS_ASSERT(value_node != nullptr);
  auto value_node = node->cast<ValueNodePtr>();
  if (value_node == nullptr) {
    MS_LOG(ERROR) << "cnode input0 is not a valuenode.";
    return lite::RET_ERROR;
  }
  MS_ASSERT(value_node->value() != nullptr);
  auto primitive_c = value_node->value()->cast<PrimitiveCPtr>();
  if (primitive_c == nullptr) {
    MS_LOG(ERROR) << "cnode has no primitive_c.";
    return lite::RET_ERROR;
  }
  auto primitive = primitive_c->primitiveT();
  if (primitive == nullptr) {
    MS_LOG(ERROR) << "cnode has no schema::primitive.";
    return lite::RET_ERROR;
  }
  if (primitive->value.type != schema::PrimitiveType_Resize) {
    MS_LOG(DEBUG) << "cnode is not cast node.";
    return RET_OK;
  }
  auto value = primitive->value.value;
  if (value == nullptr) {
    MS_LOG(ERROR) << "value is nullptr.";
    return lite::RET_ERROR;
  }
  auto attr = reinterpret_cast<schema::ResizeT *>(value);
  if (cnode->inputs().size() > 3 && attr->coordinateTransformMode == schema::CoordinateTransformMode_TF_CROP_AND_RESIZE) {
    auto new_resize_inputs = cnode->inputs();
    new_resize_inputs.erase(new_resize_inputs.begin()+1);
    cnode->set_inputs(new_resize_inputs);
  }
  return lite::RET_OK;
}

In [ ]:
/**
 * Copyright 2020 Huawei Technologies Co., Ltd
 *
 * Licensed under the Apache License, Version 2.0 (the "License");
 * you may not use this file except in compliance with the License.
 * You may obtain a copy of the License at
 *
 * http://www.apache.org/licenses/LICENSE-2.0
 *
 * Unless required by applicable law or agreed to in writing, software
 * distributed under the License is distributed on an "AS IS" BASIS,
 * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
 * See the License for the specific language governing permissions and
 * limitations under the License.
 */
#include "src/runtime/kernel/arm/fp32/nonzero_fp32.h"
#include <vector>
#include "schema/model_generated.h"
#include "src/kernel_registry.h"
#include "src/tensor.h"
#include "nnacl/fp32/nonzero_fp32.h"
#include "nnacl/op_base.h"
#include "src/runtime/runtime_api.h"
#include "include/errorcode.h"

using mindspore::kernel::KERNEL_ARCH::kCPU;
using mindspore::lite::KernelRegistrar;
using mindspore::lite::RET_ERROR;
using mindspore::lite::RET_OK;
using mindspore::schema::PrimitiveType_NonZero;

namespace mindspore::kernel {
int NonZeroCPUKernel::Init() {
  if (!InferShapeDone()) {
    return RET_OK;
  }
  return ReSize();
}

int NonZeroCPUKernel::ReSize() {
  return RET_OK;
}
int NonZeroCPUKernel::Run() {
  auto in_tensor = in_tensors_.front();
  auto out_tensor = out_tensors_.front();
  auto input_data = reinterpret_cast<float *>(in_tensor->MutableData());
  auto output_data = reinterpret_cast<int *>(out_tensor->MutableData());
  auto input_dim_size = in_tensor->shape().size();
  if (out_tensor->shape().size() != 2) {
    MS_LOG(ERROR) << "out tensor shape size must be equal to 2!";
    return RET_ERROR;
  }
  auto non_zero_nums = out_tensor->shape()[1];
  int non_zero_count = 0;
  std::vector coordiate_values(in_tensor->shape().size(), 0);
  for (size_t i = 0; i < in_tensor->ElementsNum(); i += 1) {
    if (input_data[i] != 0) {
      for (size_t j = 0; j < input_dim_size; j++) {
        output_data[non_zero_count + j * non_zero_nums] = coordiate_values[j];
      }
      non_zero_count++;
    }
    for (size_t idx = input_dim_size - 1; idx >= 0; --idx) {
      if (coordiate_values[idx] != in_tensor->shape()[idx] - 1) {
        coordiate_values[idx]++;
        break;
      }
      coordiate_values[idx] = 0;
    }
  }
  return RET_OK;
}

kernel::LiteKernel *CpuNonZeroFp32KernelCreator(const std::vector<lite::Tensor *> &inputs,
                                                const std::vector<lite::Tensor *> &outputs, OpParameter *opParameter,
                                                const lite::InnerContext *ctx, const kernel::KernelKey &desc,
                                                const mindspore::lite::PrimitiveC *primitive) {
  if (opParameter == nullptr) {
    MS_LOG(ERROR) << "Input opParameter is nullptr!";
    return nullptr;
  }
  if (ctx == nullptr) {
    MS_LOG(ERROR) << "Input context is nullptr!";
    free(opParameter);
    return nullptr;
  }
  if (ctx->thread_num_ == 0) {
    MS_LOG(ERROR) << "context thread num is 0!";
    free(opParameter);
    return nullptr;
  }
  auto *kernel = new(std::nothrow) NonZeroCPUKernel(opParameter, inputs, outputs, ctx, primitive);
  if (kernel == nullptr) {
    MS_LOG(ERROR) << "new NonZeroCPUKernel fail!";
    free(opParameter);
    return nullptr;
  }
  auto ret = kernel->Init();
  if (ret != RET_OK) {
    MS_LOG(ERROR) << "Init kernel failed, name: " << opParameter->name_ << ", type: "
                  << schema::EnumNamePrimitiveType(static_cast<schema::PrimitiveType>(opParameter->type_));
    delete kernel;
    return nullptr;
  }
  return kernel;
}

REG_KERNEL(kCPU, kNumberTypeFloat32, PrimitiveType_NonZero, CpuNonZeroFp32KernelCreator)
REG_KERNEL(kCPU, kNumberTypeBool, PrimitiveType_NonZero, CpuNonZeroFp32KernelCreator)
}  // namespace mindspore::kernel


In [ ]:
/**
 * Copyright 2020 Huawei Technologies Co., Ltd
 *
 * Licensed under the Apache License, Version 2.0 (the "License");
 * you may not use this file except in compliance with the License.
 * You may obtain a copy of the License at
 *
 * http://www.apache.org/licenses/LICENSE-2.0
 *
 * Unless required by applicable law or agreed to in writing, software
 * distributed under the License is distributed on an "AS IS" BASIS,
 * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
 * See the License for the specific language governing permissions and
 * limitations under the License.
 */
#ifndef MINDSPORE_LITE_SRC_RUNTIME_KERNEL_ARM_FP32_NONZERO_H_
#define MINDSPORE_LITE_SRC_RUNTIME_KERNEL_ARM_FP32_NONZERO_H_

#include <vector>
#include "src/lite_kernel.h"
#include "nnacl/fp32/nonzero_fp32.h"

namespace mindspore::kernel {
class NonZeroCPUKernel : public LiteKernel {
 public:
  NonZeroCPUKernel(OpParameter *parameter, const std::vector<lite::Tensor *> &inputs,
                   const std::vector<lite::Tensor *> &outputs, const lite::InnerContext *ctx,
                   const mindspore::lite::PrimitiveC *primitive)
      : LiteKernel(parameter, inputs, outputs, ctx, primitive) {}

  ~NonZeroCPUKernel() = default;

  int Init() override;
  int ReSize() override;
  int Run() override;
 protected:
  int thread_count_ = 1;
};
}  // namespace mindspore::kernel

#endif  // MINDSPORE_LITE_SRC_RUNTIME_KERNEL_ARM_FP32_NONZERO_H_


In [ ]:
/**
 * Copyright 2019-2020 Huawei Technologies Co., Ltd
 *
 * Licensed under the Apache License, Version 2.0 (the "License");
 * you may not use this file except in compliance with the License.
 * You may obtain a copy of the License at
 *
 * http://www.apache.org/licenses/LICENSE-2.0
 *
 * Unless required by applicable law or agreed to in writing, software
 * distributed under the License is distributed on an "AS IS" BASIS,
 * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
 * See the License for the specific language governing permissions and
 * limitations under the License.
 */

#include "src/ops/nonzero.h"
#include <algorithm>
#include "include/errorcode.h"
#include "src/common/log_adapter.h"
#include "src/tensor.h"
#ifndef PRIMITIVE_WRITEABLE
#include "src/ops/ops_register.h"
#endif

namespace mindspore {
namespace lite {
#ifdef PRIMITIVE_WRITEABLE
int NonZero::UnPackAttr(const Primitive &prim, const std::vector<AnfNodePtr> &inputs) {
  if (this->primitive_ == nullptr) {
    this->primitive_ = new (std::nothrow) schema::PrimitiveT;
    if (this->primitive_ == nullptr) {
      MS_LOG(ERROR) << "new primitiveT failed";
      return RET_ERROR;
    }
    this->primitive_->value.type = schema::PrimitiveType_NonZero;
  }
  if (this->primitive_->value.type != schema::PrimitiveType_NonZero) {
    MS_LOG(ERROR) << "Primitive type is error :" << this->primitive_->value.type;
    return RET_ERROR;
  }
  if (this->primitive_->value.value == nullptr) {
    this->primitive_->value.value = new (std::nothrow) schema::NonZeroT();
    if (this->primitive_->value.value == nullptr) {
      MS_LOG(ERROR) << "new primitiveT value failed";
      return RET_ERROR;
    }
  }
  PopulaterQuantParam(prim, inputs);
  return RET_OK;
}
#else
int NonZero::UnPackToFlatBuilder(const schema::Primitive *primitive, flatbuffers::FlatBufferBuilder *fbb) {
  MS_ASSERT(nullptr != primitive);
  MS_ASSERT(nullptr != fbb);
  auto attr = primitive->value_as_NonZero();
  if (attr == nullptr) {
    MS_LOG(ERROR) << "value_as_Merge return nullptr";
    return RET_ERROR;
  }
  auto val_offset = schema::CreateNonZero(*fbb);
  auto prim_offset = schema::CreatePrimitive(*fbb, schema::PrimitiveType_NonZero, val_offset.o);
  fbb->Finish(prim_offset);
  return RET_OK;
}
PrimitiveC *NonZeroCreator(const schema::Primitive *primitive) { return PrimitiveC::NewPrimitiveC<NonZero>(primitive); }
Registry NonZeroRegistry(schema::PrimitiveType_NonZero, NonZeroCreator);
#endif
template<typename T>
void CalShape(const T *data, const std::vector<Tensor *> &inputs, std::vector<int> *out_shape) {
  int input_count = inputs[0]->ElementsNum();
  int input_dim_size = inputs[0]->shape().empty() ? 1 : inputs[0]->shape().size();
  (*out_shape)[0] = input_dim_size;
  int nonzero_size = 0;
  for (int i = 0; i < input_count; i++) {
    if (static_cast<int>(data[i]) != 0) {
      nonzero_size++;
    }
  }
  if (nonzero_size == 0) {
    *out_shape = {};
  } else {
    (*out_shape)[1] = nonzero_size / input_dim_size;
  }
}
int NonZero::InferShape(std::vector<Tensor *> inputs_, std::vector<Tensor *> outputs_) {
  MS_ASSERT(this->primitive_ != nullptr);
  MS_ASSERT(inputs_.size() == 1);
  auto input = inputs_.front();
  MS_ASSERT(input != nullptr);
  auto output = outputs_.front();
  MS_ASSERT(output != nullptr);
  output->set_data_type(input->data_type());
  output->set_format(input->format());
  if (!infer_flag()) {
    return RET_INFER_INVALID;
  }

  std::vector<int> out_shape;
  if (inputs_.size() == kSingleNum) {
    auto input_tensor = inputs_.at(0);
    if (input_tensor->data_c() == nullptr) {
      MS_LOG(INFO) << "Do infer shape in runtime.";
      return RET_INFER_INVALID;
    }
    size_t shape_size = input_tensor->ElementsNum();
    switch (input_tensor->data_type()) {
      case kNumberTypeInt8: {
        auto data = reinterpret_cast<int8_t *>(input_tensor->MutableData());
        CalShape<int8_t>(data, inputs_, &out_shape);
      }
        break;
      case kNumberTypeInt32: {
        auto data = reinterpret_cast<int32_t *>(input_tensor->MutableData());
        CalShape<int32_t>(data, inputs_, &out_shape);
      }
        break;
      case kNumberTypeInt64: {
        auto data = reinterpret_cast<int64_t *>(input_tensor->MutableData());
        CalShape<int64_t>(data, inputs_, &out_shape);
      }
        break;
      case kNumberTypeFloat: {
        auto data = reinterpret_cast<float *>(input_tensor->MutableData());
        CalShape<float>(data, inputs_, &out_shape);
      }
        break;
      case kNumberTypeUInt32: {
        auto data = reinterpret_cast<uint32_t *>(input_tensor->MutableData());
        CalShape<uint32_t>(data, inputs_, &out_shape);
      }
      case kNumberTypeBool: {
        auto data = reinterpret_cast<uint32_t *>(input_tensor->MutableData());
        CalShape<uint32_t>(data, inputs_, &out_shape);
      }
        break;
      default: {
        MS_LOG(ERROR) << "NonZero weight tensor has unsupported dataType: " << input_tensor->data_type();
        return RET_INFER_ERR;
      }
    }
  } else {
    MS_LOG(ERROR) << "inputs tensor size invalid.";
    return RET_INFER_ERR;
  }
  output->set_shape(out_shape);
  return RET_OK;
}
}  // namespace lite
}  // namespace mindspore


In [ ]:
/**
 * Copyright 2019-2020 Huawei Technologies Co., Ltd
 *
 * Licensed under the Apache License, Version 2.0 (the "License");
 * you may not use this file except in compliance with the License.
 * You may obtain a copy of the License at
 *
 * http://www.apache.org/licenses/LICENSE-2.0
 *
 * Unless required by applicable law or agreed to in writing, software
 * distributed under the License is distributed on an "AS IS" BASIS,
 * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
 * See the License for the specific language governing permissions and
 * limitations under the License.
 */

#ifndef MINDSPORE_LITE_SRC_OPS_NONZERO_H_
#define MINDSPORE_LITE_SRC_OPS_NONZERO_H_

#include <vector>
#include <set>
#include <cmath>
#include <memory>

#include "src/ops/primitive_c.h"

namespace mindspore {
namespace lite {
class NonZero : public PrimitiveC {
 public:
  NonZero() = default;
  ~NonZero() = default;
#ifdef PRIMITIVE_WRITEABLE
  MS_DECLARE_PARENT(NonZero, PrimitiveC);
  explicit NonZero(schema::PrimitiveT *primitive) : PrimitiveC(primitive) {}
  int UnPackAttr(const Primitive &prim, const std::vector<AnfNodePtr> &inputs) override;
#else
  int UnPackToFlatBuilder(const schema::Primitive *primitive, flatbuffers::FlatBufferBuilder *fbb) override;
#endif
  int InferShape(std::vector<lite::Tensor *> inputs_, std::vector<lite::Tensor *> outputs_) override;
};
}  // namespace lite
}  // namespace mindspore

#endif  // MINDSPORE_LITE_SRC_OPS_NONZERO_H_
